## Deliverable 2. Create a Customer Travel Destinations Map.

In [1]:
# Dependencies and Setup
import pandas as pd
import requests
import gmaps

# Import API key
from config import g_key

# Configure gmaps API key
gmaps.configure(api_key=g_key)

In [4]:
# 1. Import the WeatherPy_database.csv file. 
city_data_df = pd.read_csv("../Weather_Database/WeatherPy_Database.csv")
city_data_df.head()

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
0,0,Punta Arenas,CL,-53.1500,-70.9167,57.31,47,40,36.82,scattered clouds
1,1,Hermanus,ZA,-34.4187,19.2345,76.89,79,94,10.13,overcast clouds
2,2,Dawlatabad,AF,36.4141,64.9053,46.31,61,100,8.68,overcast clouds
3,3,Novikovo,RU,51.1537,37.8879,27.45,95,84,8.52,broken clouds
4,4,Esperance,AU,-33.8667,121.9000,65.10,73,99,14.99,overcast clouds


In [5]:
# 2. Prompt the user to enter minimum and maximum temperature criteria 
min_temp = float(input("What is the minimum temperature you would like for your trip? "))
max_temp = float(input("What is the maximum temperature you would like for your trip? "))

What is the minimum temperature you would like for your trip? 75
What is the maximum temperature you would like for your trip? 90


In [6]:
# 3. Filter the city_data_df DataFrame using the input statements to create a new DataFrame using the loc method.
preferred_cities_df = city_data_df.loc[(city_data_df["Max Temp"] <= max_temp) &
                                       (city_data_df["Max Temp"] >= min_temp)]
preferred_cities_df.head(10)

,City_ID,City,Country,Lat,Lng,Max Temp,Humidity,Cloudiness,Wind Speed,Current Description
1,1,Hermanus,ZA,-34.4187,19.2345,76.89,79,94,10.13,overcast clouds
9,9,Tiarei,PF,-17.5333,-149.3333,75.22,76,99,12.21,light rain
15,15,Hithadhoo,MV,-0.6000,73.0833,81.93,66,76,8.75,broken clouds
16,16,Puerto Baquerizo Moreno,EC,-0.9000,-89.6000,79.79,78,40,12.66,scattered clouds
19,19,Georgetown,MY,5.4112,100.3354,83.89,74,20,1.99,few clouds
21,21,Faanui,PF,-16.4833,-151.7500,79.74,76,71,13.49,light rain
24,24,Rikitea,PF,-23.1203,-134.9692,78.93,70,63,13.98,light rain
27,27,Grand Gaube,MU,-20.0064,57.6608,81.00,71,75,3.00,broken clouds
28,28,Aswan,EG,24.0934,32.9070,75.63,20,0,18.79,clear sky
30,30,East London,ZA,-33.0153,27.9116,83.34,78,20,13.80,few clouds


In [12]:
# 4a. Determine if there are any empty rows.
preferred_cities_df.isnull().sum()

City_ID                0
City                   0
Country                2
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [15]:
# 4b. Drop any empty rows and create a new DataFrame that doesn’t have empty rows.
clean_df = preferred_cities_df.dropna()
clean_df.isnull().sum()

City_ID                0
City                   0
Country                0
Lat                    0
Lng                    0
Max Temp               0
Humidity               0
Cloudiness             0
Wind Speed             0
Current Description    0
dtype: int64

In [16]:
# 5a. Create DataFrame called hotel_df to store hotel names along with city, country, max temp, and coordinates.
hotel_df = clean_df[["City", "Country", "Max Temp", "Current Description", "Lat", "Lng"]].copy()

# 5b. Create a new column "Hotel Name"
hotel_df["Hotel Name"] = ""
hotel_df.head(10)

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hermanus,ZA,76.89,overcast clouds,-34.4187,19.2345,
9,Tiarei,PF,75.22,light rain,-17.5333,-149.3333,
15,Hithadhoo,MV,81.93,broken clouds,-0.6000,73.0833,
16,Puerto Baquerizo Moreno,EC,79.79,scattered clouds,-0.9000,-89.6000,
19,Georgetown,MY,83.89,few clouds,5.4112,100.3354,
21,Faanui,PF,79.74,light rain,-16.4833,-151.7500,
24,Rikitea,PF,78.93,light rain,-23.1203,-134.9692,
27,Grand Gaube,MU,81.00,broken clouds,-20.0064,57.6608,
28,Aswan,EG,75.63,clear sky,24.0934,32.9070,
30,East London,ZA,83.34,few clouds,-33.0153,27.9116,


In [17]:
# 6a. Set parameters to search for hotels with 5000 meters.
params = {
    "radius": 5000,
    "type": "lodging",
    "key": g_key
}

# 6b. Iterate through the hotel DataFrame.
for index, row in hotel_df.iterrows():
    # 6c. Get latitude and longitude from DataFrame
    lat = row["Lat"]
    lng = row["Lng"]
    
    # Add the latitude and longitude to location key for the params dictionary.
    params["location"] = f"{lat},{lng}"
    
    # 6d. Set up the base URL for the Google Directions API to get JSON data.
    base_url = "https://maps.googleapis.com/maps/api/place/nearbysearch/json"

    # 6e. Make request and retrieve the JSON data from the search. 
    hotels = requests.get(base_url, params=params).json()
    
    # 6f. Get the first hotel from the results and store the name, if a hotel isn't found skip the city.
    try:
        hotel_df.loc[index, "Hotel Name"] = hotels["results"][0]["name"]
    except (IndexError):
        print("Hotel not found... skipping.")

# Indicate that Data Loading is complete.
print("-----------------------------")
print("Data Retrieval Complete      ")
print("-----------------------------")

Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.
Hotel not found... skipping.


In [36]:
# 7. Drop the rows where there is no Hotel Name.
nan_value = float("NaN")
hotel_df.replace("", nan_value, inplace=True)
hotel_df.dropna(subset = ["Hotel Name"], inplace=True)
hotel_df

,City,Country,Max Temp,Current Description,Lat,Lng,Hotel Name
1,Hermanus,ZA,76.89,overcast clouds,-34.4187,19.2345,Misty Waves Boutique Hotel
9,Tiarei,PF,75.22,light rain,-17.5333,-149.3333,Le Rocher de Tahiti
15,Hithadhoo,MV,81.93,broken clouds,-0.6000,73.0833,Scoop Guest House
16,Puerto Baquerizo Moreno,EC,79.79,scattered clouds,-0.9000,-89.6000,Hotel los Algarrobos
19,Georgetown,MY,83.89,few clouds,5.4112,100.3354,Cititel Penang
...,...,...,...,...,...,...,...
663,Salalah,OM,77.09,few clouds,17.0151,54.0924,"HAMDAN PLAZA HOTEL SALALAH, an HTG Hotel"
671,Sinnamary,GF,79.86,broken clouds,5.3833,-52.9500,ALETHEIA Appartement Sinnamary
676,Fomboni,KM,80.42,moderate rain,-12.2800,43.7425,Auberge les abou
678,Hambantota,LK,77.34,scattered clouds,6.1241,81.1185,Bungalow 63


In [35]:
# 8a. Create the output File (CSV)
output_data_file = "WeatherPy_vacation.csv"
# 8b. Export the City_Data into a csv
hotel_df.to_csv(output_data_file, index_label="City_ID")

In [45]:
# 9. Using the template add city name, the country code, the weather description and maximum temperature for the city.
info_box_template = """
<dl>
<dt>Hotel Name</dt><dd>{Hotel Name}</dd>
<dt>City</dt><dd>{City}</dd>
<dt>Country</dt><dd>{Country}</dd>
<dt>Current Weather</dt><dd>{Current Description} and {Max Temp} °F</dd>
</dl>
"""

# 10a. Get the data from each row and add it to the formatting template and store the data in a list.
hotel_info = [info_box_template.format(**row) for index, row in hotel_df.iterrows()]

# 10b. Get the latitude and longitude from each row and store in a new DataFrame.
locations = hotel_df[["Lat", "Lng"]]

In [42]:
# 11a. Add a marker layer for each city to the map. 
locations = hotel_df[["Lat", "Lng"]]
max_temp = hotel_df["Max Temp"]
fig = gmaps.figure(center=(30.0, 31.0), zoom_level=1.5)
heat_layer = gmaps.heatmap_layer(locations, weights=max_temp,dissipating=False,
             max_intensity=300, point_radius=4)
marker_layer = gmaps.marker_layer(locations, info_box_content=hotel_info)
fig.add_layer(heat_layer)
fig.add_layer(marker_layer)

# 11b. Display the figure
fig

Figure(layout=FigureLayout(height='420px'))